In [1]:
import os
from pathlib import Path
import pandas as pd
import numpy as np
import pymysql

# ===== 설정 =====
ROOT_DIR = r"C:\Users\user\Documents\서울시_포폴\서울시분기별매출"
TABLE = "stg_seoul_sales_quarter"

MYSQL = dict(
    host="localhost",
    user="root",
    password="********",
    db="seoul",
    port=3306,
    charset="utf8mb4",
    autocommit=False
)

CHUNK_SIZE = 20000

# ===== 유틸 =====
def read_csv_auto(path):
    for enc in ("utf-8-sig", "cp949", "euc-kr"):
        try:
            return pd.read_csv(path, encoding=enc)
        except Exception:
            pass
    raise ValueError(f"인코딩 실패: {path}")

def normalize_cols(df):
    # 컬럼명 좌우 공백 제거
    df.columns = df.columns.astype(str).str.strip()

    # ~ 를 _ 로 통일 (시간대_00~06 → 시간대_00_06)
    rename = {}
    for c in df.columns:
        c2 = c.replace("~", "_")
        # 건수 컬럼이 '시간대_건수~06_매출_건수'처럼 이상하게 들어온 경우가 있어 보여서 정리
        c2 = c2.replace("시간대_건수_", "시간대_")
        rename[c] = c2
    return df.rename(columns=rename)

def to_int_cols(df, exclude):
    df = df.copy()
    for c in df.columns:
        if c in exclude:
            continue
        # 숫자 문자열에 콤마가 있을 가능성 대비
        df[c] = (df[c].astype(str)
                 .str.replace(",", "", regex=False)
                 .str.strip())
        df[c] = pd.to_numeric(df[c], errors="coerce")
    return df

def get_insert_sql(cols):
    col_sql = ",".join([f"`{c}`" for c in cols])
    placeholders = ",".join(["%s"] * len(cols))
    return f"INSERT IGNORE INTO {TABLE} ({col_sql}) VALUES ({placeholders})"

def load_sales_files():
    conn = pymysql.connect(**MYSQL)

    files = sorted(Path(ROOT_DIR).glob("seoul_sales_*.csv"))
    if not files:
        raise ValueError("seoul_sales_*.csv 파일을 못 찾았어. 경로 확인해줘.")

    # DB 컬럼 순서 (DDL 기준)
    cols = [
        "기준_년분기_코드","행정동_코드","행정동_코드_명","서비스_업종_코드","서비스_업종_코드_명",
        "당월_매출_금액","당월_매출_건수",
        "주중_매출_금액","주말_매출_금액","월요일_매출_금액","화요일_매출_금액","수요일_매출_금액",
        "목요일_매출_금액","금요일_매출_금액","토요일_매출_금액","일요일_매출_금액",
        "시간대_00_06_매출_금액","시간대_06_11_매출_금액","시간대_11_14_매출_금액",
        "시간대_14_17_매출_금액","시간대_17_21_매출_금액","시간대_21_24_매출_금액",
        "남성_매출_금액","여성_매출_금액",
        "연령대_10_매출_금액","연령대_20_매출_금액","연령대_30_매출_금액","연령대_40_매출_금액",
        "연령대_50_매출_금액","연령대_60_이상_매출_금액",
        "주중_매출_건수","주말_매출_건수","월요일_매출_건수","화요일_매출_건수","수요일_매출_건수",
        "목요일_매출_건수","금요일_매출_건수","토요일_매출_건수","일요일_매출_건수",
        "시간대_00_06_매출_건수","시간대_06_11_매출_건수","시간대_11_14_매출_건수",
        "시간대_14_17_매출_건수","시간대_17_21_매출_건수","시간대_21_24_매출_건수",
        "남성_매출_건수","여성_매출_건수",
        "연령대_10_매출_건수","연령대_20_매출_건수","연령대_30_매출_건수","연령대_40_매출_건수",
        "연령대_50_매출_건수","연령대_60_이상_매출_건수",
        "source_file"
    ]

    sql = get_insert_sql(cols)

    ok, fail = 0, 0
    with conn.cursor() as cur:
        for f in files:
            path = str(f)
            print(f"\n[START] {f.name}")

            try:
                df = read_csv_auto(path)
                df = normalize_cols(df)

                # 필수 컬럼 체크
                required = {"기준_년분기_코드","행정동_코드","서비스_업종_코드"}
                missing = required - set(df.columns)
                if missing:
                    raise ValueError(f"필수 컬럼 누락: {missing}")

                # 문자열 컬럼 제외하고 숫자 변환
                exclude = {"기준_년분기_코드","행정동_코드_명","서비스_업종_코드","서비스_업종_코드_명"}
                df = to_int_cols(df, exclude=exclude)

                # 행정동_코드가 숫자인지(문자면 숫자화)
                df["행정동_코드"] = pd.to_numeric(df["행정동_코드"], errors="coerce")
                df = df.dropna(subset=["기준_년분기_코드","행정동_코드","서비스_업종_코드"])

                df["행정동_코드"] = df["행정동_코드"].astype(int)

                df["source_file"] = f.name

                # NaN -> None
                sub = df[cols].astype(object).where(pd.notnull(df[cols]), None)
                data = [tuple(r) for r in sub.to_numpy()]

                for i in range(0, len(data), CHUNK_SIZE):
                    cur.executemany(sql, data[i:i+CHUNK_SIZE])
                    print(f"  inserted {min(i+CHUNK_SIZE, len(data))}/{len(data)}")

                conn.commit()
                ok += 1
                print(f"[DONE] {f.name} rows={len(df)}")

            except Exception as e:
                conn.rollback()
                fail += 1
                print(f"[FAIL] {f.name} -> {e}")

    conn.close()
    print("\n===================================")
    print(f"files ok={ok}, fail={fail}")
    print("ALL DONE")

# 실행
load_sales_files()



[START] seoul_sales_2023.csv
[FAIL] seoul_sales_2023.csv -> "['시간대_00_06_매출_건수', '시간대_06_11_매출_건수', '시간대_11_14_매출_건수', '시간대_14_17_매출_건수', '시간대_17_21_매출_건수', '시간대_21_24_매출_건수'] not in index"

[START] seoul_sales_2024.csv
[FAIL] seoul_sales_2024.csv -> "['시간대_00_06_매출_건수', '시간대_06_11_매출_건수', '시간대_11_14_매출_건수', '시간대_14_17_매출_건수', '시간대_17_21_매출_건수', '시간대_21_24_매출_건수'] not in index"

[START] seoul_sales_20251.csv
[FAIL] seoul_sales_20251.csv -> 필수 컬럼 누락: {'행정동_코드', '기준_년분기_코드', '서비스_업종_코드'}

[START] seoul_sales_20252.csv
[FAIL] seoul_sales_20252.csv -> 필수 컬럼 누락: {'행정동_코드', '기준_년분기_코드', '서비스_업종_코드'}

files ok=0, fail=4
ALL DONE
